In [ ]:
new one 

In [307]:
# Medical Lab Report Parser for Jupyter Notebook
# Run each cell sequentially

# Cell 1: Import required libraries
import pandas as pd
import re
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [309]:
# Cell 2: Load your OCR text data
# Replace this with your actual OCR text
ocr_text = """
ZERO BLOCK
Patient Profile ›
Patient Name
Age/Sex
City
Mobile No.
Blood Test Reports for Diagnosis Done By
Report Done
Before
After
ZEROX
BLOCK
Diabetic Friendly
Products of
Senera® Essentials
A Division of Tanmay Endeavours (P) Ltd.
005
Mr. Umrao Singh
50 Y/M
Dewas (MP)
9425036725
Cholesterol
Thyrocare, CHL Hospital
Before and After Taking ZERO BLOCK/ ZERO BLOCK X
Report Done on Date: 13 March 2020
Report Done on Date: 10 August 2020
Result

Before
After
Total Cholesterol
249.04
213 mg/dl
HDL
41.65
42 mg/d T
Triglyceride
214.42
207 mg/d J
LDL
164.5
140 mg/d J
VLDL

- mg/dl
LDL/ HDL Ratio
3.9
3.3 Ratio
Total Cholesterol/ HDL Ratio
5.9
5.0 Ratio
This is to inform you that all above reports are diagnosed by verified pathology labs and the other details are not tempered and as actual sent by consumers themselves, voluntarily.
We are only taking creative freedom to show these data. For display purpose only, not in use for any legal claims.



# 2nd file 

TANUSH DIAGNOSTICS
Complete Pathology Solution
DR. VIVEK J. DUGAD
M.B.B.S., M.D. (Path)
Patient Name : MR. UMESH JADHAV
Referral : Dr. Dudhedia A. P.
Report Date : Sep 27, 2021, 01:14 p.m.
Age: 32 years (Male)
Reg. ID: - 24405
Report ID : 91498
Test Description
LIPID PROFILE
Value(s)
Reference Range
S.Cholesterol
111.1 mg/di
Desirable
: < 200
Borderline high : 200 - 239
Undesirable : > 240
S.Triglycerides
169.9 mg/dl
Desirable : < 150
Borderline high : 150 - 199
High
: 200 - 499
Very High
: > 500
S. HDL
63.8 mg/dl
Undesirable : < 40
Borderline
: 40 - 59
Desirable
: > 60
S. LDL
79.4 mg/di
Desirable
:< 130
Borderline
:130-159
Undesirable
: > 160
S. VLDL
33.98 mg/di
5-51
LDLC/HDLC Ratio
1.24
0 - 3.6
S. Cholesterol/HDLC Ratio
1.74
0 - 5.0
**END OF REPORT**
Viveke
Dr. Vivek J. Dugad
M.B.B.S., M.D. ( PATH )
Reg. No. 2007/04/0619
® Plot no 373, Sector no. 24, Opp. Dominos Pizza, Tilak Road, Pradikaran, Nigdi, Pune - 411 044. Tel : 020 - 2765 5339
9 Graund floor, Lifeline Hospital Complex, Sector No. 1, Near Hotel Haveli, Indrayani Nagar, Bhosari, Pune - 411 039. Mob : 9423832565


"""

# If you have the OCR text in a file, uncomment and modify this:
# with open('your_ocr_file.txt', 'r', encoding='utf-8') as f:
#     ocr_text = f.read()

print(f"📄 OCR text loaded. Length: {len(ocr_text)} characters")
    

📄 OCR text loaded. Length: 2058 characters


In [313]:
# Cell 3: Define Master Test List
MASTER_TEST_LIST = [
    'Total Cholesterol', 'HDL Cholesterol', 'LDL Cholesterol', 'VLDL Cholesterol',
    'Triglycerides', 'S.Cholesterol', 'HDL', 'LDL', 'VLDL', 'TG',
    'Glucose', 'Blood Sugar', 'FBS', 'PPBS', 'Random Blood Sugar',
    'HbA1c', 'Glycated Hemoglobin', 'Hemoglobin A1c',
    'TSH', 'T3', 'T4', 'Free T3', 'Free T4', 'Thyroid Stimulating Hormone',
    'Creatinine', 'Urea', 'BUN', 'Blood Urea Nitrogen',
    'SGOT', 'SGPT', 'ALT', 'AST', 'Bilirubin', 'Total Bilirubin',
    'Hemoglobin', 'Hb', 'RBC', 'WBC', 'Platelet Count', 'Hematocrit',
    'ESR', 'CRP', 'C-Reactive Protein',
    'Vitamin D', 'Vitamin B12', 'Folate', 'Iron', 'Ferritin',
    'Calcium', 'Phosphorus', 'Magnesium', 'Sodium', 'Potassium', 'Chloride',
    'Protein', 'Albumin', 'Globulin', 'A/G Ratio'
]



print(f"🧪 Master test list created with {len(MASTER_TEST_LIST)} tests")


🧪 Master test list created with 58 tests


In [316]:
def clean_text(text: str) -> str:
    """Clean OCR artifacts from text."""
    if not text or not isinstance(text, str):
        return ""
    # Remove common OCR artifacts
    text = re.sub(r'[|]{2,}', '', text)  # Remove multiple pipes
    text = re.sub(r'\n+', '\n', text)    # Collapse multiple newlines
    text = re.sub(r'\s+', ' ', text)     # Normalize whitespace
    return text.strip()

def segment_reports(text: str) -> List[str]:
    """Split the master text into individual lab reports."""
    if not text or not isinstance(text, str):
        print("⚠️  Warning: No valid text provided for segmentation")
        return []
    
    # Clean the text first
    text = clean_text(text)
    
    if len(text) < 10:
        print("⚠️  Warning: Text too short for processing")
        return []
    
    # Report boundary indicators
    report_indicators = [
        r'Patient\s+Name\s*:',
        r'Patient\s+Profile\s*:',
        r'PATHKIND',
        r'THYROCARE',
        r'SRL',
        r'LAB\s+REPORT',
        r'MEDICAL\s+REPORT',
        r'TEST\s+REPORT'
    ]
    
    # Find all potential report start positions
    boundaries = []
    for indicator in report_indicators:
        try:
            matches = list(re.finditer(indicator, text, re.IGNORECASE))
            boundaries.extend([m.start() for m in matches])
        except Exception as e:
            print(f"⚠️  Error with pattern {indicator}: {e}")
            continue
    
    # Sort boundaries and remove duplicates
    boundaries = sorted(set(boundaries))
    
    if not boundaries:
        # If no clear boundaries, treat as single report
        print("ℹ️  No report boundaries found, treating as single report")
        return [text]
    
    # Split text at boundaries
    reports = []
    for i in range(len(boundaries)):
        start = boundaries[i]
        end = boundaries[i + 1] if i + 1 < len(boundaries) else len(text)
        report_text = text[start:end].strip()
        if len(report_text) > 50:  # Filter out very short segments
            reports.append(report_text)
    
    return reports

print("🔧 Helper functions defined successfully!")

🔧 Helper functions defined successfully!


In [321]:
# Cell 5: Patient Information Extraction
def extract_patient_info(report_text: str) -> Dict[str, Optional[str]]:
    """Extract patient information from a single report."""
    patient_info = {'name': None, 'age': None, 'sex': None}
    
    # Patterns for patient name
    name_patterns = [
        r'Patient\s+Name\s*:\s*([^\n\r]+)',
        r'Name\s*:\s*([^\n\r]+)',
        r'Patient\s*:\s*([^\n\r]+)'
    ]
    
    # Patterns for age
    age_patterns = [
        r'Age\s*:\s*(\d+)',
        r'Age\s+(\d+)',
        r'(\d+)\s*Y(?:ears?|rs?)?',
        r'(\d+)\s*y'
    ]
    
    # Patterns for sex
    sex_patterns = [
        r'Sex\s*:\s*(Male|Female|M|F)',
        r'Gender\s*:\s*(Male|Female|M|F)',
        r'\b(Male|Female|M|F)\b'
    ]
    
    # Extract name
    for pattern in name_patterns:
        match = re.search(pattern, report_text, re.IGNORECASE)
        if match:
            name = match.group(1).strip()
            # Clean name of extra characters
            name = re.sub(r'[^\w\s.-]', '', name).strip()
            if name and len(name) > 1:
                patient_info['name'] = name
                break
    
    # Extract age
    for pattern in age_patterns:
        match = re.search(pattern, report_text, re.IGNORECASE)
        if match:
            try:
                age = int(match.group(1))
                if 0 <= age <= 150:  # Reasonable age range
                    patient_info['age'] = age
                    break
            except ValueError:
                continue
    
    # Extract sex
    for pattern in sex_patterns:
        match = re.search(pattern, report_text, re.IGNORECASE)
        if match:
            sex = match.group(1).upper()
            if sex in ['M', 'MALE']:
                patient_info['sex'] = 'Male'
            elif sex in ['F', 'FEMALE']:
                patient_info['sex'] = 'Female'
            break
    
    return patient_info

print("👤 Patient extraction function ready!")


👤 Patient extraction function ready!


In [326]:
print("👤 Patient extraction function ready!")

# Cell 6: Test Result Parsing Functions
def parse_test_line(line: str, all_lines: List[str], line_idx: int, test_name: str) -> Optional[Dict]:
    """Parse a specific test line to extract result, unit, and reference range."""
    
    # Common patterns for test results
    # Pattern 1: Test Name Result Unit Range (all in one line)
    pattern1 = rf'{re.escape(test_name)}\s*:?\s*(\d+\.?\d*)\s*([a-zA-Z/%]+)?\s*(.*?)$'
    
    # Pattern 2: More flexible extraction
    # Look for numbers that could be results
    number_pattern = r'(\d+\.?\d*)'
    unit_pattern = r'([a-zA-Z/%]+)'
    range_pattern = r'(<?\s*\d*\.?\d*\s*-?\s*>?\s*\d*\.?\d*|<\s*\d+\.?\d*|>\s*\d+\.?\d*)'
    
    match = re.search(pattern1, line, re.IGNORECASE)
    if match:
        result_str = match.group(1)
        unit = match.group(2) if match.group(2) else ''
        range_info = match.group(3).strip() if match.group(3) else ''
        
        try:
            result_value = float(result_str)
            
            # Clean up range information
            range_match = re.search(range_pattern, range_info)
            ref_range = range_match.group(0) if range_match else range_info
            
            return {
                'test_name': test_name,
                'result': result_value,
                'unit': unit.strip(),
                'biological_ref_interval': ref_range.strip()
            }
        except ValueError:
            pass
    
    # Try to find result in the same line or next few lines
    for offset in range(0, min(3, len(all_lines) - line_idx)):
        check_line = all_lines[line_idx + offset].strip()
        numbers = re.findall(number_pattern, check_line)
        
        if numbers:
            try:
                result_value = float(numbers[0])
                
                # Extract unit and range from surrounding text
                unit_match = re.search(unit_pattern, check_line)
                unit = unit_match.group(0) if unit_match else ''
                
                range_match = re.search(range_pattern, check_line)
                ref_range = range_match.group(0) if range_match else ''
                
                return {
                    'test_name': test_name,
                    'result': result_value,
                    'unit': unit,
                    'biological_ref_interval': ref_range
                }
            except ValueError:
                continue
    
    return None

👤 Patient extraction function ready!


In [329]:
def extract_test_results(report_text: str) -> List[Dict]:
    """Extract test results from a single report."""
    results = []
    lines = report_text.split('\n')
    
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue
        
        # Check if this line contains any test from our master list
        for test_name in MASTER_TEST_LIST:
            # Create flexible pattern for test matching
            test_pattern = re.escape(test_name).replace(r'\ ', r'\s*')
            if re.search(rf'\b{test_pattern}\b', line, re.IGNORECASE):
                result_data = parse_test_line(line, lines, i, test_name)
                if result_data:
                    results.append(result_data)
                break
    
    return results

print("🧬 Test result parsing functions ready!")


🧬 Test result parsing functions ready!


In [332]:
def process_all_reports(ocr_text: str) -> pd.DataFrame:
    """
    Parse multi-page OCR text containing various medical lab reports and return structured DataFrame.
    """
    print("🚀 Starting OCR text processing...")
    
    # Step 1: Segment the document into individual reports
    reports = segment_reports(ocr_text)
    print(f"📊 Identified {len(reports)} potential lab reports")
    
    # Step 2: Process each report
    all_results = []
    
    for report_id, report_text in enumerate(reports, 1):
        print(f"⚙️  Processing report {report_id}...")
        
        # Extract patient information
        patient_info = extract_patient_info(report_text)
        print(f"   👤 Patient: {patient_info.get('name', 'Unknown')}, Age: {patient_info.get('age', 'N/A')}, Sex: {patient_info.get('sex', 'Unknown')}")
        
        # Extract test results
        test_results = extract_test_results(report_text)
        print(f"   🧪 Found {len(test_results)} test results")
        
        # Combine patient info with test results
        for test_data in test_results:
            row = {
                'patient_name': patient_info.get('name', 'Unknown'),
                'age': patient_info.get('age'),
                'sex': patient_info.get('sex', 'Unknown'),
                'test_name': test_data['test_name'],
                'result': test_data['result'],
                'unit': test_data['unit'],
                'biological_ref_interval': test_data['biological_ref_interval'],
                'source_report_id': report_id
            }
            all_results.append(row)
    
    # Step 3: Create final DataFrame
    if all_results:
        df = pd.DataFrame(all_results)
        print(f"✅ Successfully extracted {len(df)} test results from {len(reports)} reports")
        return df
    else:
        print("❌ No test results could be extracted")
        # Return empty DataFrame with correct structure
        return pd.DataFrame(columns=[
            'patient_name', 'age', 'sex', 'test_name', 'result', 
            'unit', 'biological_ref_interval', 'source_report_id'
        ])

print("🎯 Main processing function ready!")

🎯 Main processing function ready!


In [335]:
print("=" * 60)
print("🔬 STARTING MEDICAL LAB REPORT ANALYSIS")
print("=" * 60)

# Process the OCR text
df_results = process_all_reports(ocr_text)

print("\n" + "=" * 60)
print("📈 ANALYSIS COMPLETE!")
print("=" * 60)

🔬 STARTING MEDICAL LAB REPORT ANALYSIS
🚀 Starting OCR text processing...
📊 Identified 3 potential lab reports
⚙️  Processing report 1...
   👤 Patient: None, Age: 50, Sex: Male
   🧪 Found 0 test results
⚙️  Processing report 2...
   👤 Patient: None, Age: None, Sex: Male
   🧪 Found 1 test results
⚙️  Processing report 3...
   👤 Patient: MR. UMESH JADHAV Referral  Dr. Dudhedia A. P. Report Date  Sep 27 2021 0114 p.m. Age 32 years Male Reg. ID - 24405 Report ID  91498 Test Description LIPID PROFILE Values Reference Range S.Cholesterol 111.1 mgdi Desirable   200 Borderline high  200 - 239 Undesirable   240 S.Triglycerides 169.9 mgdl Desirable   150 Borderline high  150 - 199 High  200 - 499 Very High   500 S. HDL 63.8 mgdl Undesirable   40 Borderline  40 - 59 Desirable   60 S. LDL 79.4 mgdi Desirable  130 Borderline 130-159 Undesirable   160 S. VLDL 33.98 mgdi 5-51 LDLCHDLC Ratio 1.24 0 - 3.6 S. CholesterolHDLC Ratio 1.74 0 - 5.0 END OF REPORT Viveke Dr. Vivek J. Dugad M.B.B.S. M.D.  PATH  

In [338]:
print("📋 SUMMARY STATISTICS:")
print(f"   • Total test results extracted: {len(df_results)}")
print(f"   • Unique patients: {df_results['patient_name'].nunique() if len(df_results) > 0 else 0}")
print(f"   • Number of reports: {df_results['source_report_id'].nunique() if len(df_results) > 0 else 0}")

if len(df_results) > 0:
    print(f"   • Test types found: {df_results['test_name'].nunique()}")
    print("\n🧪 TEST TYPES IDENTIFIED:")
    for test in sorted(df_results['test_name'].unique()):
        count = df_results[df_results['test_name'] == test].shape[0]
        print(f"   • {test}: {count} result(s)")


📋 SUMMARY STATISTICS:
   • Total test results extracted: 2
   • Unique patients: 1
   • Number of reports: 2
   • Test types found: 2

🧪 TEST TYPES IDENTIFIED:
   • Total Cholesterol: 1 result(s)
   • Triglycerides: 1 result(s)


In [341]:
print("\n📊 FIRST 10 RESULTS:")
if len(df_results) > 0:
    display(df_results.head(10))
    
    print("\n📈 DATAFRAME INFO:")
    print(df_results.info())
else:
    print("No results to display")


📊 FIRST 10 RESULTS:


,patient_name,age,sex,test_name,result,unit,biological_ref_interval,source_report_id
0,None,NaN,Male,Total Cholesterol,249.04,,213,2
1,MR. UMESH JADHAV Referral Dr. Dudhedia A. P. ...,32.0,Male,Triglycerides,169.90,mg/dl,,3



📈 DATAFRAME INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   patient_name             1 non-null      object 
 1   age                      1 non-null      float64
 2   sex                      2 non-null      object 
 3   test_name                2 non-null      object 
 4   result                   2 non-null      float64
 5   unit                     2 non-null      object 
 6   biological_ref_interval  2 non-null      object 
 7   source_report_id         2 non-null      int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 256.0+ bytes
None


In [344]:
print("\n🎉 Analysis complete! Your data is ready in the 'df_results' DataFrame.")


🎉 Analysis complete! Your data is ready in the 'df_results' DataFrame.
